In [6]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import preprocess_input

In [7]:
# Constants
INIT_LR = 1e-4
EPOCHS = 20
BS = 32


In [8]:
DIRECTORY = r'C:\Users\aisho\OneDrive\Desktop\data'
CATEGORIES = ['fake', 'real']


In [9]:
# Data loading and preprocessing
data = []
labels = []
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)  # Preprocessing specific to ResNet50
        data.append(image)
        labels.append(category)

C:\Users\aisho\anaconda3\envs\tf_new\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [10]:
# Convert labels to one-hot encoded format
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [11]:
data = np.array(data, dtype="float32")
labels = np.array(labels)


In [12]:
trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [13]:
# Data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [14]:
# Load pre-trained ResNet50 model without the top (fully connected) layers
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Construct the head of the model to be placed on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# Place the head FC model on top of the base model
model = Model(inputs=baseModel.input, outputs=headModel)


94765736/94765736 [==============================] - 76s 1us/step


In [15]:
# Freeze the base model layers
for layer in baseModel.layers:
    layer.trainable = False


In [16]:
# Compile the model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Train the model
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS
)

C:\Users\aisho\anaconda3\envs\tf_new\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/20
164/164 [==============================] - 282s 2s/step - loss: 0.6285 - accuracy: 0.6835 - val_loss: 0.4897 - val_accuracy: 0.7527
Epoch 2/20
164/164 [==============================] - 237s 1s/step - loss: 0.5072 - accuracy: 0.7542
Epoch 3/20
164/164 [==============================] - 215s 1s/step - loss: 0.4347 - accuracy: 0.8070
Epoch 4/20
164/164 [==============================] - 271s 2s/step - loss: 0.4076 - accuracy: 0.8265
Epoch 5/20
164/164 [==============================] - 333s 2s/step - loss: 0.3679 - accuracy: 0.8417
Epoch 6/20
164/164 [==============================] - 352s 2s/step - loss: 0.3534 - accuracy: 0.8535
Epoch 7/20
164/164 [==============================] - 390s 2s/step - loss: 0.3194 - accuracy: 0.8696
Epoch 8/20
164/164 [==============================] - 475s 3s/step - loss: 0.3039 - accuracy: 0.8761
Epoch 9/20
164/164 [==============================] - 451s 3s/step - loss: 0.2955 - accuracy: 0.8773
Epoch 10/20
164/164 [============================

In [ ]:
# Evaluate the model
predIdxs = model.predict(testX, batch_size=BS)
predIdxs = np.argmax(predIdxs, axis=1)
true_labels = np.argmax(testY, axis=1)

In [ ]:
# Print classification report
print(classification_report(true_labels, predIdxs, target_names=lb.classes_))


In [ ]:
# Visualize training history (loss and accuracy)
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()


In [ ]:
# Plot confusion matrix
conf_matrix = confusion_matrix(true_labels, predIdxs)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=lb.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()